In [1]:
import pandas as pd

### 0. Configuration

Feature glossary: <br />
<ul>
<li>gender -> {0, 1} = {female, male}</li>
<li>rf (risk factors) -> {0, 1} = {no, yes}</li>
<li>sbp -> systolic blood pressure</li>
<li>hr{1, 2} -> heart rate {1, 2}</li>
<li>st -> st deviation -> {0, 1} = {no, yes}</li>
<li>crt -> creatinine</li>
<li>kil -> kilip class -> {1, 2, 3, 4}</li>
<li>target -> cardiovascular event ? {0, 1} = {no, yes}</li>
</ul>

In [2]:
datasetsFolder = '../datasets'
datasetName = f'{datasetsFolder}/DATA.txt'
datasetColumns = [
    'gender', 'age', 'rf',
    'sbp', 'hr1', 'st',
    'hr2', 'crt', 'kil',
    'target'
]
heartRateStd = {'hr1': 2, 'hr2': 0.5}

### 1. Importing and pre-processing the data

#### Importing the dataset

In [3]:
dataset = pd.read_csv(datasetName, sep='   ', header=None, engine='python')
dataset.columns = datasetColumns

#### Pre-processing

In [4]:
def convertToDataType(dataset, cols, datatype):
    for col in cols:
        dataset[col] = dataset[col].astype(datatype)
    return dataset

In [5]:
dataset = convertToDataType(
    dataset, 
    ['gender', 'age', 'rf', 'st', 'kil', 'target'],
    int
)

In [6]:
# Check if clinical guidelines are
# true:
# if (crt >= 1.3 && st=1) OR (kil >= 2)
# then target should be 1
# otherwise is 0
# dataset[dataset['kil'] >= 2]['target'].sum()

Get HR estimation based on hr1 and hr2

<img src='../images/two-sensor-data-fusion-1.png'></img>
<img src='../images/two-sensor-data-fusion-2.png'></img>

In [7]:
def getEstimation(dataset, std):
    std1Squared = std['hr1'] ** 2
    std2Squared = std['hr2'] ** 2
    k1 = std2Squared / (std1Squared + std2Squared)
    _k1 = std1Squared / (std1Squared + std2Squared)
    return k1 * dataset['hr1'] + _k1 * dataset['hr2']


In [8]:
dataset['hr'] = getEstimation(dataset, heartRateStd)